In [1]:
import numpy as np
import pandas as pd
import datetime
from collections import defaultdict
import blpapiwrapper as bw
import glob
import tqdm
import datetime

%run bloomarket_data_API.ipynb

# RA FUNCS

In [59]:

def parse_bulk_data(data):
    data_dict = {}
    for el in data:
        for key in el.keys():
            if key not in data_dict.keys():
                data_dict[key] = []
            data_dict[key].append(el[key])
    tmp = pd.DataFrame.from_dict(data_dict)
    if "Date" in data_dict.keys():
        tmp.set_index(["Analyst"], inplace = True)
    return tmp

def prepare_raw_data(tickers, years=None, months=None, update_flag = False):
    if years is None and months is None:
        years = map(str, range(2006, datetime.datetime.now().date().year + 1))
        months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

    timepoints = []
    for year in years:
        for month in months:
            if len(month) == 1:
                month = '0' + month
            timepoints.append(year + month + "01")
    timepoints.append(''.join(str(datetime.datetime.now().date()).split('-')))
    
    # prepare raw RA data
    print "Prepare raw RA data"
    for ticker in tqdm.tqdm(tickers):
        df = pd.DataFrame()
        for tpoint in np.unique(timepoints):
            print tpoint
            blpts = bw.BLPTS(securities=[ticker + ' EQUITY'], 
                             fields=["BEST_ANALYST_RECS_BULK"], strOverrideField = ["END_DATE_OVERRIDE"], strOverrideValue = [tpoint])
            try:
                blpts.get()
            except ValueError:
                print "ValueError: Incompatible indexer with Series"
                continue
                
            if type(blpts.output.ix[0,0]) != float:
                push = parse_bulk_data(blpts.output.ix[0,0])
                df = df.append(push, ignore_index=True)
            #print tpoint
        read_args = {'parse_dates' : ['Date'], 'index_col' : 'Date'}
        if not update_flag:
            df.to_csv("../Features/RA_raw/" + ticker + ".csv")
        else:   
            try:
                prev_df = pd.read_csv('../Features/RA_raw/' + ticker + '.csv', **read_args)
            except:
                prev_df = pd.DataFrame()
            df = pd.concat([prev_df, df], axis=0)
            if df.shape[0] > 0:
                df.drop_duplicates(keep=False).to_csv("../Features/RA_raw/" + ticker + ".csv")
        #print df
        #break
        blpts.closeSession()


# SI factor

In [60]:
def prepare_raw_SI(sym, start_date=datetime.datetime(2006,1,1), end_date=datetime.datetime.now()):
    res = bw.simpleHistoryRequest(securities=[sym + " EQUITY"],
                                  fields = ["SHORT_INT", 'BS_SH_OUT'],
                                  startDate=start_date,
                                  endDate=end_date,
                                  periodicity='DAILY')
    df = res[sym + ' EQUITY']
    df.index.name = 'Date'
    return df

# SALES INV

In [61]:
def prepare_raw_IS(sym, start_date=datetime.datetime(2006,1,1), end_date=datetime.datetime.now()):
    res = bw.simpleHistoryRequest(securities=[sym + " EQUITY"],
                                  fields = ["INVENTORY_GROWTH_TO_SALES_GROWTH"],
                                  startDate=start_date,
                                  endDate=end_date,
                                  periodicity='DAILY')
    df = res[sym + ' EQUITY']
    df.index.name = 'Date'
    return df

# MAIN

In [73]:
read_args = {'parse_dates' : ['Date'], 'index_col' : 'Date'}
in_spx_mask = pd.read_csv('../in_spx_mask.csv', **read_args)
universe = map(str, list(in_spx_mask.columns))
market_data_na = pd.read_csv('../un_spx_daily_nonadjusted_close.csv', **read_args)
last_universe = map(str, list(market_data_na.columns))
last_update_t = market_data_na.index[-1]
now_t = datetime.datetime.now()
delta = now_t - last_update_t         # timedelta
dates_range = []    
for i in range(delta.days + 1):
    dates_range.append(last_update_t + datetime.timedelta(days=i))    

In [74]:
years_to_upd = map(str, list(set([dt.year for dt in dates_range])))
month_to_upd = map(str, list(set([dt.month for dt in dates_range])))
print(years_to_upd, month_to_upd)    

(['2017'], ['4', '5'])


In [ ]:
new_tickers = #... provide me new_tickers

##### RA FACTOR RAW

In [75]:
# update ra raw for previously known tickers
prepare_raw_data(last_universe, years_to_upd, month_to_upd, update_flag=True)

Prepare raw RA data



  0%|                                                  | 0/738 [00:00<?, ?it/s]

20170401
20170501


20170503



  0%|                                        | 1/738 [00:06<1:18:35,  6.40s/it]

...


100%|███████████████████████████████████████▉| 736/738 [39:59<00:06,  3.08s/it]

20170401
Empty response received for USG7665A1019 EQUITY
20170501
Empty response received for USG7665A1019 EQUITY
20170503
Empty response received for USG7665A1019 EQUITY



100%|███████████████████████████████████████▉| 737/738 [40:01<00:02,  2.78s/it]

20170401
20170501
20170503



100%|████████████████████████████████████████| 738/738 [40:05<00:00,  3.15s/it]


In [97]:
# get ra raw for new tickers
prepare_raw_data(new_tickers)

Prepare raw RA data


  0%|                                                    | 0/2 [00:00<?, ?it/s]

20060101
Empty response received for SG9999014823 EQUITY
20060201
Empty response received for SG9999014823 EQUITY
20060301
Empty response received for SG9999014823 EQUITY
20060401
Empty response received for SG9999014823 EQUITY
20060501
Empty response received for SG9999014823 EQUITY
20060601
Empty response received for SG9999014823 EQUITY
20060701
Empty response received for SG9999014823 EQUITY
20060801
Empty response received for SG9999014823 EQUITY
20060901
Empty response received for SG9999014823 EQUITY
20061001
Empty response received for SG9999014823 EQUITY
20061101
Empty response received for SG9999014823 EQUITY
20061201
Empty response received for SG9999014823 EQUITY
20070101
Empty response received for SG9999014823 EQUITY
20070201
Empty response received for SG9999014823 EQUITY
20070301
Empty response received for SG9999014823 EQUITY
20070401
Empty response received for SG9999014823 EQUITY
20070501
Empty response received for SG9999014823 EQUITY
20070601
Empty response receive

 50%|█████████████████████▌                     | 1/2 [02:13<02:13, 133.39s/it]

20060101
20060201
20060301
20060401
20060501
20060601
20060701
20060801
20060901
20061001
20061101
20061201
20070101
20070201
20070301
20070401
20070501
20070601
20070701
20070801
20070901
20071001
20071101
20071201
20080101
20080201
20080301
20080401
20080501
20080601
20080701
20080801
20080901
20081001
20081101
20081201
20090101
20090201
20090301
20090401
20090501
20090601
20090701
20090801
20090901
20091001
20091101
20091201
20100101
20100201
20100301
20100401
20100501
20100601
20100701
20100801
20100901
20101001
20101101
20101201
20110101
20110201
20110301
20110401
20110501
20110601
20110701
20110801
20110901
20111001
20111101
20111201
20120101
20120201
20120301
20120401
20120501
20120601
20120701
20120801
20120901
20121001
20121101
20121201
20130101
20130201
20130301
20130401
20130501
20130601
20130701
20130801
20130901
20131001
20131101
20131201
20140101
20140201
20140301
20140401
20140501
20140601
20140701
20140801
20140901
20141001
20141101
20141201
20150101
20150201
20150301
2

100%|███████████████████████████████████████████| 2/2 [04:37<00:00, 136.66s/it]


In [95]:
ra_all = map(lambda s: s[30:-4], glob.glob('../Features/RA_raw/*.csv'))
all_ = map(lambda s: s[32:-4], glob.glob('../MarketData_raw/NA/*.csv'))
[sym for sym in ra_all if sym not in all_]

['US0138171014', 'US1255771065', 'US8685361037', 'USG7665A1019']

##### MARKET DATA RAW

In [79]:
# non-adjusted part 
bulk_market_data(last_universe, '../MarketData_raw/NA/', start_date=last_update_t, end_date=datetime.datetime.now(), update_flag=True)

In [80]:
bulk_market_data(new_tickers, '../MarketData_raw/NA/')

In [81]:
# adjusted part
output_path = '../MarketData_raw/A/'
input_path = '../MarketData_raw/NA/'
from glob import glob
files = glob(input_path + '*.csv')
symbols = list(map(lambda filename: filename[32:-4], files))
symbols[:3]

['AN8068571086', 'BMG491BT1088', 'BMG6359F1032']

In [ ]:
for sym, filename in tqdm.tqdm(zip(symbols, files)):
    mdf = pd.read_csv(filename, parse_dates=['Date'], index_col='Date')
    mdf = adjust_price_to_dividends(sym, mdf)
    mdf.to_csv(output_path + '/{}.csv'.format(sym))

##### DIV YIELD

In [34]:
for sym in tqdm.tqdm(last_universe):
    new_df = dividend_yield(sym, start_date=last_update_t, end_date=datetime.datetime.now())
    prev_df = pd.read_csv('../Features/DIV_YIELD/{}.csv'.format(sym), **read_args)
    new_df = pd.concat([prev_df, new_df], axis=0)
    new_df.groupby(new_df.index).last()\
        .to_csv('../Features/DIV_YIELD/{}.csv'.format(sym))

100%|████████████████████████████████████████| 738/738 [09:06<00:00,  1.40it/s]


In [35]:
for sym in tqdm.tqdm(new_tickers):
    new_df = dividend_yield(sym)\
        .to_csv('../Features/DIV_YIELD/{}.csv'.format(sym))

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.45it/s]


##### MARKET CAP

In [36]:
for sym in tqdm.tqdm(last_universe):
    new_df = market_cap(sym, start_date=last_update_t, end_date=datetime.datetime.now())
    prev_df = pd.read_csv('../Features/MARKET_CAP/{}.csv'.format(sym), **read_args)
    new_df = pd.concat([prev_df, new_df], axis=0)
    new_df.groupby(new_df.index).last()\
        .to_csv('../Features/MARKET_CAP/{}.csv'.format(sym))

100%|████████████████████████████████████████| 738/738 [09:42<00:00,  1.38it/s]


In [37]:
for sym in tqdm.tqdm(new_tickers):
    new_df = market_cap(sym)\
        .to_csv('../Features/MARKET_CAP/{}.csv'.format(sym))

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.09it/s]


##### SI FACTOR RAW

In [39]:
for sym in tqdm.tqdm(last_universe):
    new_df = prepare_raw_SI(sym, start_date=last_update_t, end_date=datetime.datetime.now())
    prev_df = pd.read_csv('../Features/SI_raw/{}.csv'.format(sym), **read_args)
    new_df = pd.concat([prev_df, new_df], axis=0)
    new_df.groupby(new_df.index).last()\
        .to_csv('../Features/SI_raw/{}.csv'.format(sym))

100%|████████████████████████████████████████| 738/738 [09:17<00:00,  1.40it/s]


In [40]:
for sym in tqdm.tqdm(new_tickers):
    new_df = prepare_raw_SI(sym)\
        .to_csv('../Features/SI_raw/{}.csv'.format(sym))

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.33it/s]


##### IS FACTOR RAW

In [41]:
for sym in tqdm.tqdm(last_universe):
    new_df = prepare_raw_IS(sym, start_date=last_update_t, end_date=datetime.datetime.now())
    prev_df = pd.read_csv('../Features/INV_TO_SALES/{}.csv'.format(sym), **read_args)
    new_df = pd.concat([prev_df, new_df], axis=0)
    new_df.groupby(new_df.index).last()\
        .to_csv('../Features/INV_TO_SALES/{}.csv'.format(sym))

100%|████████████████████████████████████████| 738/738 [09:49<00:00,  1.14it/s]


In [42]:
for sym in tqdm.tqdm(new_tickers):
    new_df = prepare_raw_IS(sym)\
        .to_csv('../Features/INV_TO_SALES/{}.csv'.format(sym))

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.49it/s]


In [47]:
need_isin = map(lambda s: s + ' EQUITY', list(pd.read_csv('./new_tickers.txt')['sym']))

In [51]:
for sym in need_isin:
    print ISIN(sym)
#    print '{}:{}'.format(sym, ISIN(sym))

US0079031078
US0152711091
US12503M1080
US25470M1099
US26483E1001
US23355L1061
IE00BJ3V9050
US3364331070
US35906A1088
US4130861093
US45337C1027
US3666511072
US5356781063
US7244791007
US7588491032
US7547301090
US8475601097
US8716071076
US8454671095
US9170471026


In [2]:
un = map(lambda s: s[32:-4], glob.glob('../MarketData_raw/NA/*.csv'))

In [3]:
un[:3]

['AN8068571086', 'BMG491BT1088', 'BMG6359F1032']

In [9]:
f = open('../gics_sectors.csv', 'w+')
f.write('ISIN,sector\n')
for sym in tqdm.tqdm(un):
    f.write('{},{}\n'.format(sym, GICS_sector(sym)))
f.close()

  2%|▋                                        | 14/768 [00:13<11:46,  1.07it/s]

Empty response received for IE00B40K9117 EQUITY


  5%|█▊                                       | 35/768 [00:30<11:36,  1.05it/s]

Empty response received for US0008863096 EQUITY


  7%|██▋                                      | 51/768 [00:46<08:34,  1.39it/s]

...

 98%|███████████████████████████████████████▎| 754/768 [10:00<00:11,  1.18it/s]

Empty response received for US9830241009 EQUITY


 99%|███████████████████████████████████████▍| 757/768 [10:02<00:08,  1.35it/s]

Empty response received for US98385X1063 EQUITY


100%|████████████████████████████████████████| 768/768 [10:10<00:00,  1.06it/s]


In [10]:
f = open('../sectors.csv', 'w+')
f.write('ISIN,sector,cyclical\n')
for sym in un:
    f.write('{},{}\n'.format(sym, sector(sym)))
f.close()